In [1]:
from data_processor import DataProcessor
import torch 
import torch.nn as nn
from torch.optim import Adam
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 

In [2]:
file_1 = "../data/Train.csv"
file_2 = "../data/additional_data/trainRoot_edited.csv"

processor = DataProcessor(file_1, file_2, test = False, minimal = True)
x_train, x_valid, y_train, y_valid = processor.get_numpy_data(fillna = True, additional = True,
                                                                            encode = True, np_split = True, enocde_user = False,
                                                                            normalize = True, drop_ones = True)

In [3]:
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [4]:
x_train = torch.tensor(x_train).float().to(device)
x_valid = torch.tensor(x_valid).float().to(device)
y_train = torch.tensor(y_train).float().to(device)
y_valid = torch.tensor(y_valid).float().to(device)

In [88]:
batch_size = int (len(x_train)/25)
n_epochs = 100000

In [89]:
linear_regressor = nn.Linear(in_features = len(x_train[0]), out_features = 1)

In [95]:
def train(model):
    model.to(device)
    cirterion = nn.MSELoss()
    learning_rate = .02
    optimizer = Adam(model.parameters(), lr = learning_rate)
    old_loss = 590720.5
    
    for i in range (n_epochs): 
        for b in range(0, len(x_train), batch_size):
            predictions = model(x_train[b:b+batch_size])
            loss = cirterion(predictions, y_train[b:b+batch_size])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

        if i % 1000 == 0: 
            print("The loss after 1000 epoch is: ", loss.item())
            
        if i % 10000 == 0:
            learning_rate /= 2
            print("The learning rate now is: ", learning_rate)
            old_loss = loss.item()
            optimizer = Adam(model.parameters(), lr = learning_rate)
                

In [96]:
neural_regressor = nn.Sequential(
nn.Linear(in_features = len(x_train[0]), out_features = 10),
nn.ReLU(),
nn.BatchNorm1d(10),    
nn.Linear(in_features = 10, out_features = 10), 
nn.ReLU(),
nn.BatchNorm1d(10),
nn.Linear(in_features = 10, out_features = 10), 
nn.ReLU(),
nn.BatchNorm1d(10),
nn.Linear(in_features = 10, out_features = 10), 
nn.ReLU(),
nn.BatchNorm1d(10),
nn.Linear(in_features = 10, out_features = 1))

In [97]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [98]:
neural_regressor.apply(init_weights)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


Sequential(
  (0): Linear(in_features=26, out_features=10, bias=True)
  (1): ReLU()
  (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): ReLU()
  (5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): ReLU()
  (8): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=10, out_features=10, bias=True)
  (10): ReLU()
  (11): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=10, out_features=1, bias=True)
)

In [ ]:
train(neural_regressor)

The loss after 1000 epoch is:  3874074.5
The learning rate now is:  0.01
The loss after 1000 epoch is:  41056.34765625
The loss after 1000 epoch is:  761656.875
The loss after 1000 epoch is:  41267.54296875
The loss after 1000 epoch is:  44321.1484375
The loss after 1000 epoch is:  23560.890625
The loss after 1000 epoch is:  27967.0625
The loss after 1000 epoch is:  24679.720703125


In [ ]:
torch.save(neural_regressor.state_dict(), "models/working_trained_regressor_2")

In [ ]:
#params = torch.load("models/trained_regressor_1", map_location={'cuda:0': 'cpu'})
#neural_regressor.state_dict = params

In [ ]:
predictions = neural_regressor(x_train)

In [ ]:
def evaluate(model, x, y): 
    predictions = model(x).detach().cpu().numpy().reshape(-1)
    #predictions = torch.tensor([1500.]*len(y))
    y = y.cpu().numpy().reshape(-1)
    error = mean_squared_error(y, predictions)
    var = np.var(y)
    print("The MSE error is: ", error)
    print("The variance of the validation set is: ", var)
    r_2 = 1 - error / var 

    print("The model explians " ,r_2 , " of the variance in data")


In [ ]:
evaluate(neural_regressor, x_train, y_train)
evaluate(neural_regressor, x_valid, y_valid)

In [ ]:
# processor = DataProcessor("../data/Test.csv", test = True)
# x_test = processor.get_numpy_data(True, True, True, False, True)